# Formulieren en validatie

> tags: []

# Creating a form in Vaadin: data binding and validation

Learn how to use Binder to bind a data object to input fields and validate input.


We hebben de documentatie gereorganiseerd om u een meer gestroomlijnde navigatiestructuur te bieden

1.  [Documentatie](https://vaadin.com/docs/latest/)
2.  [Tutorial](https://vaadin.com/docs/latest/tutorial)
3.  Formulieren en validatie

-   [Vaadin Binder gebruiken om een formulier te maken en invoer te valideren](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#using-vaadin-binder-to-create-a-form-and-validate-input)
-   [Het bindmiddel maken](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#creating-the-binder)
-   [De contactpersoon instellen](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#setting-the-contact)
-   [Onderdeelgebeurtenissen instellen](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#setting-up-component-events)
-   [Het formulier opslaan, verwijderen en sluiten](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#saving-deleting-and-closing-the-form)

In [het hoofdstuk Een component maken](https://vaadin.com/docs/latest/tutorial/creating-a-component) hebt u de invoervelden en knoppen gemaakt die u nodig hebt voor het bewerken van contactpersonen. In dit hoofdstuk bindt u die invoer aan een object om een volledig functionele vorm met validatie te maken.`Contact`

Dit hoofdstuk heeft betrekking op:

-   Het maken van een Vaadin Binder.
    
-   Binding invoervelden.
    
-   Veldvalidatie.
    

## [](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#using-vaadin-binder-to-create-a-form-and-validate-input)Vaadin Binder gebruiken om een formulier te maken en invoer te valideren

Een formulier is een verzameling invoervelden die zijn verbonden met een gegevensmodel, een in dit geval. Formulieren valideren gebruikersinvoer en maken het gemakkelijk om een object gevuld met invoerwaarden uit de gebruikersinterface te krijgen.`Contact`

Het bindt UI-velden op naam aan gegevensobjectvelden. Het neemt bijvoorbeeld een UI-veld met de naam en wijst het toe aan het veld van het gegevensobject en het veld aan het veld, enzovoort. Dit is de reden waarom de veldnamen in en hetzelfde zijn. Vaadin gebruikt de klasse [`Binder`](https://vaadin.com/docs/latest/binding-data/components-binder) om formulieren te bouwen.`firstName``firstName``lastName``lastName``Contact``ContactForm`

<table><tbody><tr><td _msthidden="1"><p>Note</p></td><td><p>Geavanceerde Binder API</p><p _msthash="2705769" _msttexthash="17656093">Binder ondersteunt ook een <a href="https://vaadin.com/docs/latest/binding-data/components-binder-validation" _istranslated="1">geavanceerde API</a> waar u gegevensconversies en aanvullende validatieregels kunt configureren, maar voor deze toepassing is de eenvoudige API voldoende.</p><p _msthash="2705770" _msttexthash="33807592">Binder kan validatieregels gebruiken die zijn gedefinieerd op het gegevensobject in de gebruikersinterface. Dit betekent dat u dezelfde validaties kunt uitvoeren, zowel in de browser als voordat u deze opslaat in de database, zonder code te dupliceren.</p></td></tr></tbody></table>

### [](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#defining-bean-validation-rules-in-java)Bean-validatieregels definiëren in Java

U kunt gegevensvalidatieregels definiëren als Java Bean Validation-annotaties in de Java-klasse.

U kunt alle toegepaste validatieregels bekijken door te inspecteren . De validaties worden als volgt boven de velddeclaraties geplaatst:`Contact.java`

```
@Email
@NotEmpty
private String email = "";
```

## [](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#creating-the-binder)Het bindmiddel maken

Instantiateer een bindmiddel en gebruik het om de invoervelden te binden:

```
// Other fields omitted
Binder<Contact> binder = new BeanValidationBinder<>(Contact.class); 1

public ContactForm(List<Company> companies, List<Status> statuses) {
    addClassName("contact-form");
    binder.bindInstanceFields(this); 2
    // Rest of constructor omitted
}
```

1.  `BeanValidationBinder` is a that is aware of bean validation annotations. By passing it in the , you define the type of object you are binding to.`Binder``Contact.class`
    
2.  `bindInstanceFields()` matches fields in and based on their names.`Contact``ContactForm`
    

With these two lines of code, you’ve readied the UI fields to be connected to a contact. That’s the next step.

Next, you need to create a setter for the . Unlike the companies and statuses, it can change over time as a user browses through the contacts.`contact`

To do this, add the following in the class:`ContactForm`

```
public class ContactForm extends FormLayout {
    private Contact contact;

    // other methods and fields omitted

    public void setContact(Contact contact) {
        this.contact = contact; 1
        binder.readBean(contact); 2
    }
}
```

1.  Save a reference to the contact, so you can save the form values back into it later.
    
2.  Calls to bind the values from the contact to the UI fields. copies the values from the contact to an internal model; that way you don’t overwrite values if you cancel editing.`binder.readBean()``readBean()`
    

## [](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#setting-up-component-events)Setting up Component Events

Vaadin comes with an event-handling system for components. You have already used it to listen to value-change events from the filter TextField in the main view. The form component should have a similar way of informing parent components of events. The events you will fire are:

-   `SaveEvent`
    
-   `DeleteEvent`
    
-   `CloseEvent`
    

Add the following code at the end of the class to define the new events:`ContactForm`

```
// Events
public static abstract class ContactFormEvent extends ComponentEvent<ContactForm> {
  private Contact contact;

  protected ContactFormEvent(ContactForm source, Contact contact) { 1
    super(source, false);
    this.contact = contact;
  }

  public Contact getContact() {
    return contact;
  }
}

public static class SaveEvent extends ContactFormEvent {
  SaveEvent(ContactForm source, Contact contact) {
    super(source, contact);
  }
}

public static class DeleteEvent extends ContactFormEvent {
  DeleteEvent(ContactForm source, Contact contact) {
    super(source, contact);
  }

}

public static class CloseEvent extends ContactFormEvent {
  CloseEvent(ContactForm source) {
    super(source, null);
  }
}

public <T extends ComponentEvent<?>> Registration addListener(Class<T> eventType,
    ComponentEventListener<T> listener) { 2
  return getEventBus().addListener(eventType, listener);
}
```

1.  `ContactFormEvent` is a common superclass for all the events. It contains the that was edited or deleted.`contact`
    
2.  The method uses Vaadin’s event bus to register the custom event types. Select the import for if IntelliJ asks.`addListener()``com.vaadin``Registration`
    

## [](https://vaadin.com/docs/latest/tutorial/forms-and-validation/#saving-deleting-and-closing-the-form)Saving, Deleting, and Closing the Form

With the event types defined, you can now inform anyone using of relevant events.`ContactForm`

To add , , and event listeners, add the following to the class:`save``delete``close``ContactForm`

```
private Component createButtonsLayout() {
  save.addThemeVariants(ButtonVariant.LUMO_PRIMARY);
  delete.addThemeVariants(ButtonVariant.LUMO_ERROR);
  close.addThemeVariants(ButtonVariant.LUMO_TERTIARY);

  save.addClickShortcut(Key.ENTER);
  close.addClickShortcut(Key.ESCAPE);

  save.addClickListener(event -> validateAndSave()); 1
  delete.addClickListener(event -> fireEvent(new DeleteEvent(this, contact))); 2
  close.addClickListener(event -> fireEvent(new CloseEvent(this))); 3

  binder.addStatusChangeListener(e -> save.setEnabled(binder.isValid())); 4
  return new HorizontalLayout(save, delete, close);
}

private void validateAndSave() {
  try {
    binder.writeBean(contact); 5
    fireEvent(new SaveEvent(this, contact)); 6
  } catch (ValidationException e) {
    e.printStackTrace();
  }
}
```

1.  The **save** button calls the method.`validateAndSave()`
    
2.  The **delete** button fires a delete event and passes the active contact.
    
3.  The **cancel** button fires a close event.
    
4.  Validates the form every time it changes. If it is invalid, it disables the **save** button to avoid invalid submissions.
    
5.  Write the form contents back to the original .`contact`
    
6.  Fire a save event, so the parent component can handle the action.
    

Build the project and verify that it compiles. There are no visible changes yet. In the next chapter, you will connect the form to the list view to complete the first view.

D788B762-1531-4C0C-A207-BB01672A413F

[TutorialAccessing The Database](https://vaadin.com/docs/latest/tutorial/database-access)[TutorialHandling View State](https://vaadin.com/docs/latest/tutorial/handling-view-state)

**Was this page helpful?**  
Leave a comment or a question below. You can also join the [chat on Discord](https://discord.gg/MYFq5RTbBn) or [ask questions on StackOverflow](https://stackoverflow.com/questions/tagged/vaadin).
